In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
archive_train = zipfile.ZipFile("/kaggle/input/whats-cooking/train.json.zip", 'r')
archive_submission = zipfile.ZipFile("/kaggle/input/whats-cooking/test.json.zip", 'r')

In [ ]:
dataset_train = pd.read_json(archive_train.read('train.json'))
dataset_submission = pd.read_json(archive_submission.read('test.json'))
dataset_train_copy = dataset_train.copy()
dataset_submission_copy = dataset_submission.copy()

## Data Preprocessing

In [ ]:
train_dict_count = {}

for entry_list in dataset_train_copy['ingredients']:
    for entry in entry_list:
        if entry in train_dict_count.keys():
            # Increment the count by 1
            train_dict_count[entry] += 1
        else:
            train_dict_count[entry] = 1      

In [ ]:
len(train_dict_count)

In [ ]:
submission_dict_count = {}

for entry_list in dataset_submission_copy['ingredients']:
    for entry in entry_list:
        if entry in submission_dict_count.keys():
            # Increment the count by 1
            submission_dict_count[entry] += 1
        else:
            submission_dict_count[entry] = 1      

In [ ]:
len(submission_dict_count)

## Check if all ingridients in train data are in submission and if not add it with value 0 and vice versa

In [ ]:
for key in train_dict_count:
    if key not in submission_dict_count.keys():
        # Add it to submission data
        submission_dict_count[key] = 0

In [ ]:
len(train_dict_count)

In [ ]:
for cols in train_dict_count.keys():
    dataset_train_copy[cols] = np.zeros((dataset_train_copy.shape[0], 1))

In [ ]:
for cols in submission_dict_count.keys():
    dataset_submission_copy[cols] = np.zeros((dataset_submission_copy.shape[0], 1))

In [ ]:
dataset_train_copy.shape

In [ ]:
dataset_submission_copy.shape

In [ ]:
for column_name in dataset_train_copy.columns:
    if column_name not in ['id', 'cuisine', 'ingredients']:
        for index, each_entry_list in enumerate(dataset_train_copy['ingredients']):
            if column_name in each_entry_list:
                dataset_train_copy.loc[index, column_name] = 1

In [ ]:
dataset_train_copy

In [ ]:
for column_name in dataset_submission_copy.columns:
    if column_name not in ['id', 'ingredients']:
        for index, each_entry_list in enumerate(dataset_submission_copy['ingredients']):
            if column_name in each_entry_list:
                dataset_submission_copy.loc[index, column_name] = 1

In [ ]:
dataset_submission_copy

In [ ]:
dataset_submission_copy = dataset_submission_copy[dataset_train_copy.drop('cuisine', axis=1).columns]

In [ ]:
dataset_train_copy.drop(['id', 'ingredients', 'cuisine'], axis=1, inplace=True)
dataset_submission_copy.drop(['id', 'ingredients'], axis=1, inplace=True)

## Label Encoding y

In [ ]:
X = dataset_train_copy.values
y = dataset_train.iloc[:,1].values
X_submission = dataset_submission_copy.values

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(y)

## Training on Logistic Regresison Model

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X, y)

In [ ]:
y_submission = classifier.predict(X_submission)
y_submission = le.inverse_transform(y_submission)
Id = dataset_submission.iloc[:,0]
submission_dataset = pd.DataFrame({'id':Id, 'cuisine':y_submission})
submission_dataset.set_index('id', inplace=True)
submission_dataset.to_csv('Submission.csv')